In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from collections import defaultdict

In [4]:
# Fetch data from Cloud Storage
from google.cloud import storage

client = storage.Client()
eod_balance_agg = pd.read_csv(
    "gs://berkabank/production/data/04_processing/eod_balance_aggregation.csv"
)
eod_balance_agg["balance_date"] = pd.to_datetime(eod_balance_agg["balance_date"])
eod_balance_agg = eod_balance_agg.drop(columns = ['target','is_primary','account_creation_date','streak_id','low_balance_flag'])
accounts = pd.read_csv("gs://berkabank/production/data/01_raw/accounts.csv")
incidents = pd.read_csv("gs://berkabank/production/data/03_primary/incidents.csv")
backup = {
    "eod_balance_agg": eod_balance_agg.copy(),
    "accounts": accounts.copy(),
    "incidents":incidents.copy()
}

In [36]:
eod_balance_agg = eod_balance_agg.drop(columns = ['target','is_primary','account_creation_date','streak_id','low_balance_flag'])

In [6]:
incidents

,account_id,incident_date,district_id,t0,t1
0,1926,1994-01-22,37,1993-01-02,1994-01-02
1,2256,1994-01-22,48,1993-01-02,1994-01-02
2,2434,1994-01-22,17,1993-01-02,1994-01-02
3,2666,1994-01-24,49,1993-01-04,1994-01-04
4,2330,1994-01-24,54,1993-01-04,1994-01-04
...,...,...,...,...,...
793,3061,1996-12-29,76,1995-12-10,1996-12-09
794,1629,1996-12-29,10,1995-12-10,1996-12-09
795,2686,1996-12-31,26,1995-12-12,1996-12-11
796,1426,1996-12-31,40,1995-12-12,1996-12-11


In [26]:
# Customer Base Core
core = accounts.merge(incidents,how = 'left', on = ['account_id','district_id'] ).sort_values(by = 'account_creation_date')
core['target'] = ~core['incident_date'].isnull()
core = core.loc[:,['account_id','target']]


# Training Core
training_accounts = pd.DataFrame(eod_balance_agg['account_id'].unique(), columns=['account_id'])
training_core = (
    core.merge(training_accounts,on ='account_id')
)
training_core.target.value_counts()

target
False    1351
True      798
Name: count, dtype: int64

In [37]:
eod_balance_agg

,account_id,balance_date,end_of_day_balance,daily_amount_flow,n_transactions,days_since_account_creation,low_balance_streak,district_id
0,1926,1993-01-06,700.0,700.0,0.0,0,1,37
1,1926,1993-01-07,700.0,0.0,0.0,1,2,37
2,1926,1993-01-08,700.0,0.0,0.0,2,3,37
3,1926,1993-01-09,700.0,0.0,0.0,3,4,37
4,1628,1993-01-10,700.0,700.0,0.0,0,1,60
...,...,...,...,...,...,...,...,...
688114,620,1996-12-11,20000.0,0.0,9.0,297,0,26
688115,880,1996-12-11,700.0,0.0,0.0,3,4,26
688116,822,1996-12-11,21285.8,0.0,5.0,68,0,40
688117,2223,1996-12-11,17375.1,0.0,46.0,341,64,44


In [59]:
X = training_core.merge(eod_balance_agg, on = 'account_id')

In [55]:
# Versioning X based on time up do minute 
#version = '__'+datetime.datetime.now().strftime("%Y%m%d%H%M")
#X.to_csv(f'gs://berkabank/production/data/04_processing/eod_balance_training_aggregation.csv',index = False)

In [45]:
# Create possible indicators and merge them with training_core.
X.columns

Index(['account_id', 'balance_date', 'end_of_day_balance', 'daily_amount_flow',
       'n_transactions', 'days_since_account_creation', 'low_balance_streak',
       'district_id'],
      dtype='object')

In [60]:
X.set_index("account_id")[["target"]]

,target
account_id,
576,True
576,True
576,True
576,True
576,True
...,...
4375,False
880,False
880,False
